## Read FGCM data

- creation date : 2025-12-05

### Links
- doc lsst-pipelines : https://pipelines.lsst.io/v/v23_0_0/modules/lsst.fgcmcal/
- github : https://github.com/lsst/fgcmcal

In [ ]:
import pandas as pd
import textwrap
from astropy.table import Table
from astropy.table import join
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, get_sun
import astropy.units as u
from datetime import datetime,timedelta
import os
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

## Configuration

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

In [ ]:
# Rubin-LSST / Cerro Pachón
lsst = EarthLocation(lat=-30.2417*u.deg, lon=-70.7366*u.deg, height=2663*u.m)

In [ ]:
# where are stored the figures
pathfigs = "figs_FGCM03_ReadAtmParams"
prefix = "fgcm03"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
REPO_URI  = "dp2_prep"
collection = "LSSTCam/runs/DRP/20250417_20250723/d_2025_11_21/DM-53374"
strcollection = collection.replace("/","_")
strrepo = REPO_URI.replace("/","_")
suptitle = f"repo {REPO_URI}, coll = {collection}"

## Tools

In [ ]:
def solar_midnight_utc(day_mjd, location):
    """
    Retourne l'heure UTC approximative du min du Soleil (culmination la plus basse)
    pour le site donné et un jour MJD.
    """
    # Création d'une grille de temps toutes les 5 minutes sur ce jour
    t_start = Time(day_mjd, format='mjd')
    t_grid = t_start + np.arange(0, 1, 5/1440)  # 1 jour = 1440 min

    # Calcul altitude du Soleil
    altaz = AltAz(obstime=t_grid, location=location)
    sun_alt = get_sun(t_grid).transform_to(altaz).alt

    # Trouver l'heure du min
    idx_min = np.argmin(sun_alt)
    return t_grid[idx_min]  # Retourne un Time object

In [ ]:
# Palette par filtre
default_filter_colors = {
    "u_24": "tab:blue",
    "g_6":  "tab:green",
    "r_57": "tab:red",
    "i_39": "tab:orange",
    "z_20": "tab:gray",
    "y_10": "black"
}

def plot_atm_parameter(t_join, param="pwv", filter_colors=None):
    """
    Trace un paramètre atmosphérique par date, par filtre,
    avec bandes grises = nuit astronomique au site Rubin-LSST.

    Paramètres
    ----------
    t_join : astropy.Table
        Table jointe avec colonnes 'mjd', 'physicalFilter' et le paramètre choisi
    param : str
        Nom du paramètre à tracer ('pwv', 'o3', 'tau', etc.)
    filter_colors : dict, optional
        Dictionnaire {filter_name: couleur}, sinon palette par défaut
    """
    if filter_colors is None:
        filter_colors = default_filter_colors

    mjd = t_join['mjd']
    filters = t_join['physicalFilter']
    values = t_join[param]

    mask_valid = np.isfinite(values)
    dates_utc = Time(mjd, format='mjd').to_datetime()

    filter_order = list(filter_colors.keys())

    plt.figure(figsize=(18,8))

    # Scatter par filtre
    for f in filter_order:
        m = (filters == f) & mask_valid
        if np.sum(m) > 0:
            plt.scatter(dates_utc[m], values[m], s=12, alpha=0.6,
                        color=filter_colors[f], label=f)

    # Fonctions auxiliaires
    def night_astronomical_utc(day_mjd, location):
        t_start = Time(day_mjd, format='mjd')
        t_grid = t_start + np.arange(0, 1.5, 5/1440)  # 1.5 jour pour capturer la nuit complète
        altaz = AltAz(obstime=t_grid, location=location)
        sun_alt = get_sun(t_grid).transform_to(altaz).alt
        mask_night = sun_alt < -18*u.deg
        night_times = t_grid[mask_night]
        if len(night_times) == 0:
            return None, None
        return night_times[0], night_times[-1]

    # Bandes grises = nuit astronomique
    start_day = int(np.floor(mjd.min()))
    end_day   = int(np.ceil(mjd.max()))
    all_days = np.arange(start_day, end_day, 1)

    for day_mjd in all_days:
        start_night, end_night = night_astronomical_utc(day_mjd, lsst)
        if start_night is not None:
            plt.axvspan(start_night.datetime, end_night.datetime,
                        color='gray', alpha=0.05)

    # Format axe X
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d:%H'))
    # Locator pour avoir au moins 1 tick par semaine
    #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO, interval=1))
    #ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
    ax.xaxis.set_major_locator(mdates.AutoDateLocator(minticks=7, maxticks=15))
    # Tick mineur : 1 par jour
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    
    plt.xticks(rotation=45)

    plt.xlabel("Date (UTC)")
    plt.ylabel(f"{param.upper()}")
    plt.title(f"{param.upper()} vs Date (colored by filter)\nGray = astronomical night LSST")
    plt.legend(title="Filter", markerscale=1.5)
    plt.grid(True, alpha=0.3)
    plt.suptitle(suptitle)
    plt.tight_layout()
    figname =f"{pathfigs}/{prefix}_{param}"+figtype
    plt.savefig(figname)
    plt.show()

## Start

## Read file

In [ ]:
!ls data

In [ ]:
# Choix du format : "fits" ou "ecsv"
format_save = "fits"  # ou "ecsv"
filename = "data/fgcm_rdp2_prep_cLSSTCam_runs_DRP_20250417_20250723_d_2025_11_21_DM-53374_20251205_095454.fits"

t_join_loaded = Table.read(filename)


In [ ]:
t_join = t_join_loaded

In [ ]:
print(t_join.colnames)
print(t_join[:2])

In [ ]:
[name for name in t_join.colnames if "pmb" in name]

### Check the physical filters

In [ ]:
unique_filters = np.unique(t_join['physicalFilter'])
print(unique_filters)

## Plots

### Plot PWV

In [ ]:
plot_atm_parameter(t_join, param="pwv")

### Plot Ozone

In [ ]:
plot_atm_parameter(t_join, param="o3")

### Plot tau

In [ ]:
plot_atm_parameter(t_join, param="tau")

### Plot alpha

In [ ]:
plot_atm_parameter(t_join, param="alpha")